In [1]:
import pandas as pd
import numpy as np
import time
import random
import lightgbm as lgb
import time

import pickle

In [2]:
FOLDER = '../../data_kaggle/champs/'
OUTPUT = FOLDER + 'out/'

In [3]:
pred_train_name = '20190711_predict_train.pickle'
pred_val_name = '20190711_predict_val.pickle'
temp_pickle = 'features_temp_{}.pickle'
temp_pickle_val = 'features_temp_val_{}.pickle'
temp_no = 0

In [4]:
df_train = pd.read_pickle(OUTPUT + pred_train_name)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]
df_val = pd.read_pickle(OUTPUT + pred_val_name)[['id', 'molecule_name', 'atom_index_0','atom_index_1','type', 'scalar_coupling_constant','predict']]

mols_train = df_train['molecule_name'].unique()
mols_val = df_val['molecule_name'].unique()

df = pd.concat([df_train, df_val], axis=0).sort_values('id')
df_idx = df.set_index('molecule_name')

In [5]:
bond = ['1JHC', '2JHC', '3JHC']

div = len(mols_train) // 8

df_train_features = pd.DataFrame()
start0 = time.time()

for m in mols_train[div*temp_no:div*(temp_no+1)]:
    if type(df_idx.loc[m]) == pd.Series:
        continue
    df_temp = df_idx.loc[m].query('type in {}'.format(bond))
    con_id = df_temp['id'].values
    df_temp_idx = df_temp.set_index('id')

    featured_sc = np.zeros([0,4])
    for idx in con_id:
        pred_cols = 'predict'
        
        pred = df_temp_idx.loc[idx][pred_cols]
        
        focus_atom = df_temp_idx.loc[idx]['atom_index_1']

        mean_1JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[0], focus_atom))\
                        [pred_cols].mean()

        mean_2JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[1], focus_atom))\
                        [pred_cols].mean()

        mean_3JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[2], focus_atom))\
                        [pred_cols].mean()

        means = np.array([pred, mean_1JHC, mean_2JHC, mean_3JHC]).reshape(-1,4)
        featured_sc = np.vstack([featured_sc, means])

    df_feature = pd.DataFrame(featured_sc)
    df_feature['id'] = con_id
    df_train_features = pd.concat([df_train_features, df_feature], axis=0)
    
df_train_features.to_pickle(OUTPUT + temp_pickle.format(temp_no))
elapsed_time = time.time() - start0
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

KeyboardInterrupt: 

In [ ]:
bond = ['1JHC', '2JHC', '3JHC']
temp_pickle_val = 'features_temp_val_{}.pickle'

div = len(mols_val) // 8

df_val_features = pd.DataFrame()
start0 = time.time()

for m in mols_val[div*temp_no:div*(temp_no+1)]:
    if type(df_idx.loc[m]) == pd.Series:
        continue
    df_temp = df_idx.loc[m].query('type in {}'.format(bond))
    con_id = df_temp['id'].values
    df_temp_idx = df_temp.set_index('id')

    featured_sc = np.zeros([0,4])
    for idx in con_id:
        pred_cols = 'predict'
        
        pred = df_temp_idx.loc[idx][pred_cols]
        
        focus_atom = df_temp_idx.loc[idx]['atom_index_1']

        mean_1JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[0], focus_atom))\
                        [pred_cols].mean()

        mean_2JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[1], focus_atom))\
                        [pred_cols].mean()

        mean_3JHC = \
        df_temp_idx.loc[con_id != idx]\
                        .query('type == "{}" and atom_index_1 == {}'.format(bond[2], focus_atom))\
                        [pred_cols].mean()

        means = np.array([pred, mean_1JHC, mean_2JHC, mean_3JHC]).reshape(-1,4)
        featured_sc = np.vstack([featured_sc, means])

    df_feature = pd.DataFrame(featured_sc)
    df_feature['id'] = con_id
    df_val_features = pd.concat([df_val_features, df_feature], axis=0)
    
df_val_features.to_pickle(OUTPUT + temp_pickle_val.format(temp_no))
elapsed_time = time.time() - start0
print ("total elapsed_time:{0}".format(elapsed_time) + "[sec]")

In [ ]:
# df_train_features_list = []
# for n in range(8):
# df_train_features_list.append(pd.read_pickle(OUTPUT + temp_pickle.format(n)))

# df_train_features = pd.concat(df_train_features_list, axis=0)

In [ ]:
# def type_score(y_val, y_pred):
#     return np.log(sum(np.abs(y_val- y_pred)) / len(y_val))